In [1]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
set_session(tf.Session(config=config))

/home/janek/.local/share/virtualenvs/rosetta-WKmHhL03/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import keras
import keras.layers as L
from keras.models import Model
import numpy as np
import pandas as pd

RANDOM_STATE=42
np.random.seed(RANDOM_STATE)
tf.set_random_seed(RANDOM_STATE)

In [3]:
START = '^'
END = '$'

SIZE = 10_000
LATENT_DIM = 128
EMBEDDING_DIM = 32

In [4]:
def create_equations_df(size, min_value=0, max_value=9999, operations={'+': np.add, '-': np.subtract}):
    df = pd.DataFrame()
    df['a'] = np.random.randint(low=min_value, high=max_value, size=size)
    df['b'] = np.random.randint(low=min_value, high=max_value, size=size)
    df['op'] = np.random.choice(list(operations.keys()), size)
    df['result'] = np.zeros(size, dtype='int')
    for symbol, calc in operations.items():
        df.loc[df.op == symbol, 'result'] = calc(df[df.op == symbol]['a'], df[df.op == symbol]['b'])
        
    df['input_texts'] = START + df.a.astype(str) + df.op + df.b.astype(str) + END
    df['target_texts'] = START + df.result.astype(str) + END
    return df

In [5]:
df = create_equations_df(SIZE)

In [6]:
corpus = pd.concat([df.input_texts, df.target_texts])

In [7]:
tokenizer = keras.preprocessing.text.Tokenizer(num_words=None, filters=None, char_level=True)
tokenizer.fit_on_texts(corpus)
df['input_sequences'] = tokenizer.texts_to_sequences(df.input_texts)
df['target_sequences'] = tokenizer.texts_to_sequences(df.target_texts)
nr_tokens = len(tokenizer.word_index)

In [8]:
X = keras.preprocessing.sequence.pad_sequences(df.input_sequences, padding='post')
y = keras.preprocessing.sequence.pad_sequences(df.target_sequences, padding='post')
y_one_hot = keras.utils.to_categorical(y, num_classes=nr_tokens)

max_len_input = X.shape[1]
max_len_target = y.shape[1]

In [51]:
encoder_gru = L.GRU(LATENT_DIM, return_state=True, name='encoder_gru')
decoder_gru = L.GRU(LATENT_DIM, return_sequences=True, return_state=True, name='decoder_gru')
decoder_dense = L.Dense(nr_tokens, activation='softmax', name='decoder_outputs')

shared_embedding = L.Embedding(nr_tokens, EMBEDDING_DIM, name='shared_embedding')

encoder_inputs = L.Input(shape=(max_len_input, ), dtype='int32', name='encoder_inputs')
encoder_embeddings = shared_embedding(encoder_inputs)
_, encoder_states = encoder_gru(encoder_embeddings)

decoder_inputs = L.Input(shape=(max_len_target,), dtype='float32', name='decoder_inputs')
decoder_embeddings_inputs = shared_embedding(decoder_inputs)
decoder_embeddings_outputs, _ = decoder_gru(decoder_embeddings_inputs, initial_state=encoder_states) 
decoder_outputs = decoder_dense(decoder_embeddings_outputs)

model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=decoder_outputs)

inference_encoder_model = Model(encoder_inputs, encoder_states)
    
inference_decoder_state_inputs = L.Input(shape=(LATENT_DIM, ), dtype='float32', name='inference_decoder_state_inputs')
inference_decoder_embeddings_outputs, inference_decoder_states = decoder_gru(
    decoder_embeddings_inputs,
    initial_state=inference_decoder_state_inputs
)
inference_decoder_outputs = decoder_dense(inference_decoder_embeddings_outputs)

inference_decoder_model = Model(
    [decoder_inputs, inference_decoder_state_inputs], 
    [inference_decoder_outputs, inference_decoder_states]
)

In [53]:
def decode_sequence(input_seq):
    states_value = inference_encoder_model.predict(input_seq)
    
    return states_value

In [54]:
decode_sequence(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(['^360+120$'])))

ValueError: Error when checking : expected encoder_inputs to have shape (11,) but got array with shape (9,)

In [21]:
s2s = Seq2Seq()
model = s2s.model
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_inputs (InputLayer)     (None, 7)            0                                            
__________________________________________________________________________________________________
encoder_inputs (InputLayer)     (None, 11)           0                                            
__________________________________________________________________________________________________
shared_embedding (Embedding)    multiple             448         encoder_inputs[0][0]             
                                                                 decoder_inputs[0][0]             
__________________________________________________________________________________________________
encoder_gru (GRU)               [(None, 128), (None, 61824       shared_embedding[0][0]           
__________

In [22]:
model.compile(optimizer=keras.optimizers.Adam(clipnorm=2.), loss='categorical_crossentropy')

In [23]:
model.fit([X, y], y_one_hot, validation_split=0.2, epochs=10)

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 5s 583us/step - loss: 0.8182 - val_loss: 0.0162
Epoch 2/10
8000/8000 [==============================] - 4s 448us/step - loss: 0.0070 - val_loss: 0.0031
Epoch 3/10
8000/8000 [==============================] - 4s 452us/step - loss: 0.0021 - val_loss: 0.0013
Epoch 4/10
8000/8000 [==============================] - 4s 453us/step - loss: 0.0010 - val_loss: 7.6095e-04
Epoch 5/10
8000/8000 [==============================] - 4s 453us/step - loss: 6.1475e-04 - val_loss: 4.8889e-04
Epoch 6/10
8000/8000 [==============================] - 4s 454us/step - loss: 4.0995e-04 - val_loss: 3.3840e-04
Epoch 7/10
8000/8000 [==============================] - 4s 453us/step - loss: 2.9124e-04 - val_loss: 2.4673e-04
Epoch 8/10
8000/8000 [==============================] - 4s 453us/step - loss: 2.1606e-04 - val_loss: 1.8632e-04
Epoch 9/10
8000/8000 [==============================] - 4s 452us/step - loss: 1.6539

In [ ]:
model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(['^360+120$']), padding='post', maxlen=X.shape[1]))